<a href="https://colab.research.google.com/github/roquearmas/IABD_DATA/blob/main/Prediccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción y salida del reconocimiento de acordes

Este cuaderno forma parte del TFM de Jesús Roque Armas Martín sobre reconocimiento de acordes con redes convolucionales.

Predice los acordes de una canción dada, mediante un modelo preentrenado en otro cuaderno.

Importación de las librerías necesarias

In [1]:
%pip install librosa
%pip install matplotlib
%pip install numpy
%pip install keras
%pip install opencv-python
%pip install PIL
%pip install flask_cors
%pip install keras
%pip install tensorflow
%pip install tkinter
%pip install simpleaudio
%pip install pygame

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL
ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simpleaudio: filename=simpleaudio-1.0.4-cp310-cp310-linux_x86_64.whl size=2054402 sha256=9574239caf93ac3fe6cad8d96d405d290f4fa1cc355d9a6694111d285a08143d
  Stored in directory: /root/.cache/pip/wheels/10/70/ed/8c41675109565c2c65c3ac40d20859fc9d3f93a8efbb11d1c7
Successfully built simpleaudio


In [2]:
import librosa
import librosa.display

import matplotlib.pyplot as plt
import numpy as np

import itertools
import math
import math

import tempfile
import os
import keras.models

import cv2
from PIL import Image

import os
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import simpleaudio as sa
import pygame
import time
import json
#from flask import Flask, request, jsonify
#from flask_cors import CORS



pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Librosa(https://zenodo.org/badge/6309729.svg)


Carga del modelo preentrenado

In [4]:
model = keras.models.load_model('/content/drive/MyDrive/proyecto deteccion acordes/modelo_entrenado/recon_acordes.h5')

Array de acordes soportados, en el mismo orden en que se entrenó el modelo

In [5]:
lista_acordes= ['Am', 'Bb', 'C', 'Dm', 'Bdim', 'G', 'F', 'Em']
acordes =np.array(lista_acordes)

**Tratamiento del fichero de audio**:

1.   Extraer los datos del fichero de audio, de la misma manera que cuando se entrenó el modelo

    -   División del fichero en muestras de medio segundo
    -   Normalización de la muestra a imagen en blanco y negro
    -   Reescalado de la imagen a 300 x 300 píxeles

2.   Predecir cada muestra de tiempo de la canción
3.   Generar una cadena de salida con el tiempo de inicio de la muestra y el acorde predicho, así como el porcentaje de confianza que el modelo tiene en dicha predicción



Paso la imagen a escala de grises, y la reescalo a 300 x 300 píxeles, antes de evaluarla

In [6]:
def normalizar_imagen(espectrograma):
  # Convertir a escala de grises
  imagen_gris = cv2.cvtColor(espectrograma, cv2.COLOR_BGR2GRAY)
  espectrograma_gris_3_canales = cv2.cvtColor(imagen_gris, cv2.COLOR_GRAY2BGR)
  #reescalar imagen
  imagen = cv2.resize(espectrograma_gris_3_canales, (300,300), interpolation=cv2.INTER_CUBIC)
  # imagen_gris = Image.fromarray(imagen_gris)
  return imagen


In [7]:
def procesar_audio(audio_file, progress_bar):
  umbral_confianza =80
  audio_data, sample_rate = librosa.load(audio_file)
  # Divide el audio en segmentos de 1/2 segundo y genera histogramas y periodogramas
  segment_duration = 0.5  # Duración del segmento en segundos
  num_segments = int(len(audio_data) / (sample_rate * segment_duration))
  #separar la parte armónica de la percusiva del audio de origen
  audio_harmonic, audio_percussive = librosa.effects.hpss(audio_data)
  salida = "["
  for i in range(num_segments):
      progress_bar['value'] = (i + 1) / num_segments * 100
      root.update_idletasks()
      start_time = i * segment_duration
      end_time = (i + 1) * segment_duration
      segment = audio_harmonic[int(start_time * sample_rate):int(end_time * sample_rate)]

      # Deshabilitar la visualización interactiva de las imágenes
      plt.ioff()

      # Periodograma del segmento
      plt.figure(figsize=(4, 4))
      librosa.display.specshow(librosa.power_to_db(abs(librosa.stft(segment)), ref=np.max), y_axis='log', x_axis='time', cmap='viridis')
      plt.axis('off')
      plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
      # Guardar la imagen
      # nombreimagen = os.path.join('/content/', 'imagen.png')
      # plt.savefig(nombreimagen, bbox_inches='tight', pad_inches=0)
      # Convertir la figura a un array de NumPy
      fig = plt.gcf()
      fig.canvas.draw()
      espectrograma_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
      espectrograma_array = espectrograma_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))

      # Cerrar la figura de Matplotlib
      plt.close()
      imagen=normalizar_imagen(espectrograma_array)
      # normalizar_imagen('/content/')
      #cargar imagen en X
      X = []
      X.append(imagen)
        #hacer predict con el modelo
      pred = model.predict(np.array(X))
      #añadir linea con el predict a un fichero de texto
      row = []
      if round(np.max(pred)*100,2)>umbral_confianza: #si la seguridad con la que está predicho es superior al 70%, lo muestro, si no, no
        predicho = np.argmax(pred) #clase con mejor predicción


        #fila ="{ timestamp: " + str(start_time) + ", text: '" + acordes[predicho] + " - " + str(round(np.max(pred)*100,2))+ "'}"
        #formato del texto de salida
        fila ='{ "timestamp": ' + str(start_time) + ', "text": "' + acordes[predicho] + '"}'
        if salida =='[':
          salida +=fila
        else :
          salida +=',' + fila




  # Escribir los datos en la salida de texto
  salida += "]"
  #print(salida)
  return salida, progress_bar

In [8]:
reproduciendo = False
current_time  = 0

def seleccionar_archivo_audio():
    archivo_audio = filedialog.askopenfilename(title="Seleccionar archivo de audio",
                                              filetypes=[("Archivos de audio", "*.wav;*.mp3;*.ogg")])
    if archivo_audio:
        boton_seleccionar.config(state='disabled')
        progress_bar = ttk.Progressbar(root, mode='determinate', length=300)
        progress_bar.pack(pady=10)

        # Procesar el archivo de audio y recibir la barra de progreso actualizada
        salida, progress_bar = procesar_audio(archivo_audio, progress_bar)

        # Actualizar la barra de progreso al 100%
        progress_bar['value'] = 100
        root.update_idletasks()

        global audio_file, anotaciones
        audio_file = archivo_audio

        try:
            anotaciones = json.loads(salida)
            print(anotaciones)  # Verificar el formato de las anotaciones
        except json.JSONDecodeError:
            print("Error: Las anotaciones no están en formato JSON válido.")
            return


        # Habilitar el botón de reproducción, el de selección y el de pausa
        boton_reproducir.config(state='normal')
        boton_pausa.config(state='normal')
        boton_seleccionar.config(state='normal')

        # Crear una etiqueta para mostrar información adicional
        global label_info
        label_info = tk.Label(root, text="", font=("Arial", 40))
        label_info.pack(pady=10)



def reproducir_audio():
    global reproduciendo
    reproduciendo = True

    # Inicializar pygame para la reproducción de audio
    pygame.mixer.init()
    pygame.mixer.music.load(audio_file)

    # Crear un label para mostrar las anotaciones
    label_anotaciones = tk.Label(root, text="", font=("Arial", 20))
    label_anotaciones.pack(pady=20)

     # Reproducir el audio con las anotaciones
    pygame.mixer.music.play()
    #while pygame.mixer.music.get_busy():
    while reproduciendo and pygame.mixer.music.get_pos() >= 0:
        current_time = pygame.mixer.music.get_pos() / 1000
        for anotacion in anotaciones:
            if anotacion['timestamp'] <= current_time:
                label_info.config(text=f"  {anotacion['text']}  ")

                root.update_idletasks()
        time.sleep(0.1)

    # Detener la reproducción del audio
    pygame.mixer.music.stop()
    reproduciendo = False

def pausar_reproduccion():
    global reproduciendo
    reproduciendo = False
    pygame.mixer.music.pause()



root = tk.Tk()
root.title("Extractor de acordes")
# Establecer el tamaño de la ventana
root.geometry("400x300")  # Ancho x Altura
# Cargar el tema 'clam'
style = ttk.Style()
style.theme_use('clam')
# Personalizar el estilo de la barra de progreso
style.configure('TProgressbar', background='#4CAF50')
# Crear un botón para seleccionar el archivo de audio
boton_seleccionar = tk.Button(root, text="Seleccionar archivo de audio", command=seleccionar_archivo_audio)
boton_seleccionar.pack(pady=20)

# Crear un botón para iniciar la reproducción
boton_reproducir = tk.Button(root, text="Reproducir audio", command=reproducir_audio, state='disabled')
boton_reproducir.pack(pady=20)

# Crear un botón para pausar la reproducción
boton_pausa = tk.Button(root, text="Stop", command=pausar_reproduccion, state='disabled')
boton_pausa.pack(pady=20)

root.mainloop()

TclError: no display name and no $DISPLAY environment variable